In [1]:
#!python
# Sample source code from the Tutorial Introduction in the documentation.

import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

import numpy
a = numpy.random.randn(4,2)
print a

a = a.astype(numpy.float32)

a_gpu = cuda.mem_alloc(a.size * a.dtype.itemsize)

cuda.memcpy_htod(a_gpu, a)

mod = SourceModule("""
    __global__ void doublify(float *a)
    {
      int idx = threadIdx.x + threadIdx.y*4;
      a[idx] *= 2;
    }
    """)

func = mod.get_function("doublify")
func(a_gpu, block=(4,4,1))

a_doubled = numpy.empty_like(a)
cuda.memcpy_dtoh(a_doubled, a_gpu)
print "original array:"
print a
print "doubled with kernel:"
print a_doubled

# alternate kernel invocation -------------------------------------------------

func(cuda.InOut(a), block=(4, 4, 1))
print "doubled with InOut:"
print a

# part 2 ----------------------------------------------------------------------

import pycuda.gpuarray as gpuarray
#a_gpu = gpuarray.to_gpu(numpy.random.randn(4,4).astype(numpy.float32))
a_gpu = gpuarray.to_gpu(a)
a_doubled = (2*a_gpu).get()

print "original array:"
print a_gpu
print "doubled with gpuarray:"
print a_doubled

[[ 0.25978142  0.65854823]
 [ 0.86870172 -0.90911217]
 [ 0.08670676  2.88062448]
 [ 2.34701272  0.32660132]]
original array:
[[ 0.25978142  0.65854824]
 [ 0.8687017  -0.90911216]
 [ 0.08670676  2.88062453]
 [ 2.34701276  0.32660133]]
doubled with kernel:
[[ 0.51956284  1.31709647]
 [ 1.73740339 -1.81822431]
 [ 0.17341352  5.76124907]
 [ 4.69402552  0.65320265]]
doubled with InOut:
[[ 0.51956284  1.31709647]
 [ 1.73740339 -1.81822431]
 [ 0.17341352  5.76124907]
 [ 4.69402552  0.65320265]]
original array:
[[ 0.51956284  1.31709647]
 [ 1.73740339 -1.81822431]
 [ 0.17341352  5.76124907]
 [ 4.69402552  0.65320265]]
doubled with gpuarray:
[[  1.03912568   2.63419294]
 [  3.47480679  -3.63644862]
 [  0.34682703  11.52249813]
 [  9.38805103   1.30640531]]


/home/taller-dis1/Angel/new_process/github/radsa
